# Scrape página web Congreso Visible 

####  importar librerías que se van a usar 

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import itertools

from selenium import webdriver
import time


####  sacar el listado de las 125 página donde se encuentran los proyectos de Ley 

In [ ]:
# sufijo que  de las páginas que se deben visitar para extrar las url de cada proyecto 
proyectos = 'https://congresovisible.uniandes.edu.co/proyectos-de-ley/#q=2019&page='
 
proyectos_list = []
for i in [x for x in range(1,126)]:
    proyectos_list.append(proyectos + str(i))

####  una vez se tiene la lista con las 125 urls: se crea una función que permita extraer la url de cada proyecto de ley 

In [ ]:
def sacar_links_pdl(lista):
    browser = webdriver.Chrome(executable_path=r"C:\web_drivers\chromedriver.exe")
    browser.get(lista)
    time.sleep(2) # tiempo que tarda el java en cargar la info de la web 
    html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    time.sleep(0)
    soup = BeautifulSoup(html,'lxml') # crear soup 
    
    elements = soup.find_all('a') # buscar todos los links 
   
    pl_prefix = 'https://congresovisible.uniandes.edu.co/proyectos-de-ley/'
    
    pl_sufix = re.findall('/proyectos-de-ley/(p.+?)\">', str(elements)) # extraer links que coinciden con este inicio 
    
    pl_all_list = [pl_prefix + url for url in pl_sufix] 
    
    return pl_all_list 

#### se aplica la función a la lista de url´s previamente creada 

In [ ]:
url_proyectos_ley = list(map(sacar_links_pdl,proyectos_list)) # resultado: 625 urls, una para cada proyecto de ley 

###  Algoritmo que extrae todos los elementos que se requieren de la pag web 

In [ ]:
def scrap_elementos(url):
       
    try: 
        get_url = requests.get(url).text
        sopa = BeautifulSoup(get_url,'lxml')
        
        r_titulo = sopa.find_all('h1')
        try:
            titulo = re.findall('\">"|“(.+?)\[', str(r_titulo))[-1]
        except:
            titulo = titulo = re.findall('\">“(.+?)\.', str(r_titulo))
        
        try:
            tagtitulo = re.findall('\[([A-Z\d].+?)\]', str(r_titulo))[-1]
               
        except:
            tagtitulo = 'NA'
        del r_titulo
        
        r_estado = sopa.find('div', class_='module5').find_all('li', text='')
        estado = re.findall('\">(.+?)</a>', str(r_estado))[-1]
        proceso = re.findall('\">(.+?)</a>', str(r_estado))
        fechas_proceso = re.findall('<p>(.+?)</p>', str(r_estado))
        del r_estado
        
        try:
            r_sinopsis = sopa.find('div', class_='module7').find('p', text='')
            sinopsis = re.findall('p>(.+?)</', str(r_sinopsis))
            sinopsis = [w.replace('<br/>', ' ') for w in sinopsis][-1]
            del r_sinopsis
        except:
            sinopsis = 'NA'
        
        r_datos_grls = sopa.find('ul', class_='lista6').find_all('li', text='')
        str_datos_grls = str(r_datos_grls)
        del r_datos_grls
        
        temas = re.findall('Tema</h3><p>(.+?)</p>', str_datos_grls)
        temas = Na_taguer(temas)
        tipo = re.findall('Tipo</h3><p>(.+?)</p>', str_datos_grls)[-1]
        tipo = Na_taguer(tipo)
        
        fecha_radicacion = re.findall('Fecha de Radicación </h3><p>(.+?)</p>', str_datos_grls)[-1]
        
        tags = re.findall('#q=(.+?)\">', str_datos_grls)
        tags = [x.strip(' ') for x in tags]
        tags = Na_taguer(tags)
        
        r_autores = sopa.find('div', class_='module2')
        autores = re.findall('alt=\"([A-ZÁÉÍÓÚ].+?)\" height', str(r_autores))
        autores = Na_taguer(autores)
        del r_autores
        
        r_partido_autores = sopa.find_all('div', class_='module2')
        partido_autores1 = re.findall('\d/\">(.+?)</p>', str(r_partido_autores))
        partido_autores = re.findall('\d/\">(.+?)</a>', str(partido_autores1))
        partido_autores = Na_taguer(partido_autores)
        del r_partido_autores
        del partido_autores1
        
        url_Pdl = url
    
        return{'Proyecto de Ley':titulo,'Palabra Clave':tagtitulo,'Estado':estado,'Autor':autores,'Partido':partido_autores ,'Proceso':proceso,'Fechas proceso':fechas_proceso,'Resumen':sinopsis,'Tema':temas,'Tipo':tipo,'Fecha Radicación':fecha_radicacion,'Tags':tags,'link':url_Pdl}
# se retorna un diccionario con los elementos requeridos 
    except:  # en caso de que algún link no funcione, para que no se detenga toda la aplciación por un error 
        
        titulo = 'link dañado'
        tagtitulo = 'link dañado'
        estado = 'link dañado'
        autores = 'link dañado'
        partido_autores = 'link dañado'
        proceso = 'link dañado'
        fechas_proceso = 'link dañado'
        sinopsis = 'link dañado'
        temas = 'link dañado'
        tipo = 'link dañado'
        fecha_radicacion = 'link dañado'
        tags = 'link dañado'
        url_Pdl = url
        
        return{'Proyecto de Ley':titulo,'Palabra Clave':tagtitulo,'Estado':estado,'Autor':autores,'Partido':partido_autores,'Proceso':proceso,'Fechas proceso':fechas_proceso,'Resumen':sinopsis,'Tema':temas,'Tipo':tipo,'Fecha Radicación':fecha_radicacion,'Tags':tags,'link':url}


####  se aplica la anterior función a cada url de cada uno de los 625 proyectos de Ley 

In [ ]:
Macro_Dicc = list(map(scrap_elementos,url_625_PdL))


#####  Se crea el DataFrame  de la MacroBase 

In [ ]:
Macro_DF = pd.DataFrame(Macro_Dicc)